# Visit job links and scan for their titles to confirm availability
Last Updated on 2023-04-07

In [ ]:
# install package: pip3 install pyairtable
import os
from dotenv import load_dotenv
from pyairtable import Api, Base, Table
from pyairtable.formulas import match
import pandas as pd
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \AppleWebKit/537.36 (KHTML, like Gecko) \Chrome/90.0.4430.212 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})

In [ ]:
# functions to retrieve job posting data
def get_soup(url,HEADERS):
    r = requests.get(url, headers=HEADERS).text
#     soup = BeautifulSoup(r, 'html.parser')
    soup = BeautifulSoup(r, 'lxml')
    return soup

In [ ]:
# load environment file and variables
load_dotenv("airtable.env")
AIRTABLE_TOKEN = os.getenv("Airtable_token")
AIRTABLE_BASE_ID = os.getenv("Airtable_ft_base_id")

In [ ]:
table = Table(AIRTABLE_TOKEN, AIRTABLE_BASE_ID, 'Jobs-2023')

In [ ]:
def check_status(title,url):
    soup= get_soup(url,HEADERS)
    # search for job title in the posting data to determine it's availability
#     pattern = re.compile(title)
    if soup.find(text=title) or title in str(soup):
        return True
    else:
        return False

In [ ]:
IDs = []

In [ ]:
counter = 0
for records in table.iterate(page_size=10, max_records=1000):
    for record in records:        
        if record['fields']['Status'] == 'Not Applied':
            title = record['fields']['Job']
            ID = record['id']
            url = record['fields']['Link']
            if check_status(ID,url) is False and ID not in IDs:
                IDs.append(ID)
                print(ID,url)
                counter +=1
print(counter)

In [ ]:
len(IDs)

In [ ]:
def update_to_NA(ID):
    table.update(ID, {"Status": 'No longer active'})

In [ ]:
update_to_NA('recNrkw17z3GURSS3') 